In [1]:
# <-- Import libraries, custom functions, and load configuration & datasets <-- #

import yaml
import datetime as dt
import re
import pandas as pd
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import seaborn as sns

from statsmodels.multivariate.manova import MANOVA
from scipy import stats
from scipy.stats import pearsonr, boxcox, chi2_contingency, shapiro, probplot
from scipy.stats.contingency import association

sns.set(style="whitegrid")

# <-- Imports custom preprocessing functions from 'functions.py' <-- #

# from functions import ()

# <-- Loads YAML configuration to dynamically reference CSV output files. <-- #

config = None  # <-- Initialize config
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")


# df_demo = pd.read_csv(config['output_data']['file1'])
df_demo_variation = pd.read_csv(config['output_data']['file2'])
df_demo_test = pd.read_csv(config['output_data']['file3'])
df_demo_control = pd.read_csv(config['output_data']['file4'])
df_web_data = pd.read_csv(config['output_data']['file5'])


In [2]:
# ========= Config =========
STEP_ORDER = ['start','step_1','step_2','step_3','confirm']
STEP_MAP   = {s:i for i,s in enumerate(STEP_ORDER)}
KEY = ['client_id','visitor_id','visit_id']   # clave del proceso

def _to_utc(series: pd.Series) -> pd.Series:
    s = pd.to_datetime(series, errors='coerce')
    if getattr(s.dt, "tz", None) is None:
        return s.dt.tz_localize("UTC")
    return s.dt.tz_convert("UTC")

def _standardize_steps(df):
    w = df.copy()
    w['process_step'] = (w['process_step'].str.lower().str.strip()
                         .str.replace(' ', '_')
                         .str.replace('step1','step_1')
                         .str.replace('step2','step_2')
                         .str.replace('step3','step_3'))
    return w[w['process_step'].isin(STEP_ORDER)]


In [ ]:
# ========= 1) Filtrado al grupo control y normalización =========
def prepare_web_for_group(df_web_data, df_demo_group):
    ids = set(df_demo_group['client_id'].unique())
    w = df_web_data[df_web_data['client_id'].isin(ids)].copy()
    w = _standardize_steps(w)
    w['date_time'] = _to_utc(w['date_time'])
    # Orden **por proceso**
    w = w.sort_values(KEY + ['date_time']).reset_index(drop=True)
    return w

In [4]:
# ========= 2) Secuencia completa → step-backs =========
def compute_back_jumps(w_full: pd.DataFrame) -> pd.DataFrame:
    wf = w_full.copy()
    wf['step_idx'] = wf['process_step'].map(STEP_MAP).astype('Int64')
    wf['prev_step_idx'] = wf.groupby(KEY)['step_idx'].shift(1)
    wf['delta'] = wf['step_idx'] - wf['prev_step_idx']
    wf['is_back_jump'] = wf['delta'].lt(0)
    back = (wf.groupby(KEY)
              .agg(n_back_jumps=('is_back_jump','sum'))
              .reset_index())
    return wf, back

In [5]:
# ========= 3) Últimas ocurrencias por paso y último confirm =========
def collapse_last_per_step_and_last_confirm(wf: pd.DataFrame) -> pd.DataFrame:
    # eliminamos confirms intermedios por proceso (deja solo el último)
    is_last_confirm = (wf['process_step'].eq('confirm') &
                       ~wf.duplicated(subset=KEY+['process_step'], keep='last'))
    drop_mask = wf['process_step'].eq('confirm') & (~is_last_confirm)
    wf2 = wf.loc[~drop_mask].copy()

    # para tiempos/completion tomamos **última** ocurrencia de cada paso por proceso
    wf2 = (wf2.sort_values(KEY + ['process_step','date_time'])
              .drop_duplicates(subset=KEY+['process_step'], keep='last'))
    return wf2

In [6]:
# ========= 4) Tabla por proceso (1 fila = 1 proceso) =========
def summarize_processes(wf2: pd.DataFrame, back: pd.DataFrame) -> pd.DataFrame:
    # Pivot de timestamps
    pv = wf2.pivot_table(index=KEY, columns='process_step', values='date_time', aggfunc='last') \
            .reindex(columns=STEP_ORDER)
    for c in STEP_ORDER:
        if c not in pv.columns:
            pv[c] = pd.NaT
    # todos como naive UTC para restas
    for c in pv.columns:
        pv[c] = _to_utc(pv[c]).dt.tz_convert('UTC').dt.tz_localize(None)

    # flags reached y completed
    flags = pv.notna().astype(int).add_prefix('reached_')
    flags['completed'] = flags['reached_confirm']

    # tiempos (min)
    def mins(b,a): return (pv[b] - pv[a]) / pd.Timedelta(minutes=1)
    times = pd.DataFrame(index=pv.index)
    times['t_start_step1'] = mins('step_1','start')
    times['t_step1_step2'] = mins('step_2','step_1')
    times['t_step2_step3'] = mins('step_3','step_2')
    times['t_step3_conf']  = mins('confirm','step_3')
    times['t_total']       = mins('confirm','start')

    proc = (flags.join(times)).reset_index()
    proc = proc.merge(back, on=KEY, how='left').fillna({'n_back_jumps':0})

    # Outcome:
    # - successful: completed y sin step-backs
    # - completed_with_errors: completed con step-backs
    # - unsuccessful: no completed
    proc['outcome'] = np.where(
        (proc['completed']==1) & (proc['n_back_jumps']==0), 'successful',
        np.where((proc['completed']==1) & (proc['n_back_jumps']>0), 'completed_with_errors', 'unsuccessful')
    )
    return proc

In [7]:
# ========= 5) KPIs & Drop-off =========
def kpis_from_processes(proc: pd.DataFrame) -> pd.DataFrame:
    d = proc.copy()
    denom = max(1, int(d['reached_start'].sum()))   # quienes empezaron

    kpis = {
        'n_processes'          : int(len(d)),
        'started'              : int(d['reached_start'].sum()),
        'step1_rate_%'         : 100*d['reached_step_1'].sum()/denom,
        'step2_rate_%'         : 100*d['reached_step_2'].sum()/denom,
        'step3_rate_%'         : 100*d['reached_step_3'].sum()/denom,
        'completion_rate_%'    : 100*d['completed'].sum()/denom,
        'successful_%'         : 100*(d['outcome'].eq('successful').sum())/denom,
        'completed_with_errors_%': 100*(d['outcome'].eq('completed_with_errors').sum())/denom,
        'unsuccessful_%'       : 100*(d['outcome'].eq('unsuccessful').sum())/denom,
        't_total_median_min'   : float(np.nanmedian(d['t_total'])),
        't_step1_median_min'   : float(np.nanmedian(d['t_start_step1'])),
        't_step2_median_min'   : float(np.nanmedian(d['t_step1_step2'])),
        't_step3_median_min'   : float(np.nanmedian(d['t_step2_step3'])),
        't_conf_median_min'    : float(np.nanmedian(d['t_step3_conf'])),
        'avg_back_jumps'       : float(d['n_back_jumps'].mean())
    }
    return pd.DataFrame([kpis])

In [8]:
def step_dropoff_table(proc: pd.DataFrame) -> pd.DataFrame:
    d = proc.copy()
    rows = [
        ("start→step_1", int(d['reached_start'].sum()),   int(d['reached_step_1'].sum())),
        ("step_1→step_2", int(d['reached_step_1'].sum()), int(d['reached_step_2'].sum())),
        ("step_2→step_3", int(d['reached_step_2'].sum()), int(d['reached_step_3'].sum())),
        ("step_3→confirm", int(d['reached_step_3'].sum()), int(d['completed'].sum())),
    ]
    out = pd.DataFrame(rows, columns=['from_to','n_from','n_to'])
    out['conv_rate_%'] = 100 * out['n_to'] / out['n_from'].replace({0:np.nan})
    out['conv_rate_%'] = out['conv_rate_%'].fillna(0.0)
    out['dropoff_%'] = 100 - out['conv_rate_%']
    return out

In [14]:
# ========= 6) RUN (CONTROL) =========
w_control = prepare_web_for_group(df_web_data, df_demo_control)          # filtra y normaliza
wf, back = compute_back_jumps(w_control)                                  # secuencia completa → step-backs
wf2 = collapse_last_per_step_and_last_confirm(wf)                         # colapsa a última ocurrencia por paso
proc_control = summarize_processes(wf2, back)                             # 1 fila por proceso + outcome
kpis_control = kpis_from_processes(proc_control)                          # KPIs
dropoff_control = step_dropoff_table(proc_control)

display(proc_control.head(15))
display(kpis_control)
display(dropoff_control)

# Sanity checks con los ejemplos que pasaste:
display("Client 4079649 processes:", 
      proc_control[proc_control['client_id']==4079649][KEY+['completed','n_back_jumps','outcome','t_total']])

display("Client 4033851 processes:", 
      proc_control[proc_control['client_id']==4033851][KEY+['completed','n_back_jumps','outcome','t_total']])

display("Client 5305116 processes:", 
      proc_control[proc_control['client_id']==5305116][KEY+['completed','n_back_jumps','outcome','t_total']])

,client_id,visitor_id,visit_id,reached_start,reached_step_1,reached_step_2,reached_step_3,reached_confirm,completed,t_start_step1,t_step1_step2,t_step2_step3,t_step3_conf,t_total,n_back_jumps,outcome
0,1028,42237450_62128060588,557292053_87239438319_391157,1,1,1,1,0,0,8.966667,-0.150000,-2.216667,NaN,NaN,2,unsuccessful
1,1186,446844663_31615102958,507052512_11309370126_442139,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,unsuccessful
2,1186,446844663_31615102958,795373564_99931517312_810896,1,1,1,0,0,0,0.183333,0.183333,NaN,NaN,NaN,0,unsuccessful
3,1195,766842522_69992551638,393817425_39015278493_996341,1,1,1,1,1,1,0.550000,1.633333,0.516667,1.383333,4.083333,0,successful
4,1197,753759429_54481946928,71862471_21202285428_848395,1,1,1,1,1,1,0.066667,1.133333,0.100000,0.283333,1.583333,1,completed_with_errors
5,1368,366307863_19014662045,784065271_45379483290_309335,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,unsuccessful
6,2439,607208067_70160939111,848231744_22569944243_37711,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,unsuccessful
7,2581,770616558_80928163524,182925466_27021409208_83502,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,unsuccessful
8,3647,742136411_92170694666,492965200_82386590930_211259,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,unsuccessful
9,5354,233679354_36495397627,192604721_14077630095_819213,1,1,1,1,1,1,0.216667,0.133333,0.283333,0.866667,1.500000,0,successful


,n_processes,started,step1_rate_%,step2_rate_%,step3_rate_%,completion_rate_%,successful_%,completed_with_errors_%,unsuccessful_%,t_total_median_min,t_step1_median_min,t_step2_median_min,t_step3_median_min,t_conf_median_min,avg_back_jumps
0,23360,22932,75.30089,64.412175,58.869702,49.598814,38.121402,11.477411,52.267574,4.316667,0.35,0.366667,1.1,1.45,0.260574


,from_to,n_from,n_to,conv_rate_%,dropoff_%
0,start→step_1,22932,17268,75.300890,24.699110
1,step_1→step_2,17268,14771,85.539727,14.460273
2,step_2→step_3,14771,13500,91.395302,8.604698
3,step_3→confirm,13500,11374,84.251852,15.748148


'Client 4079649 processes:'

,client_id,visitor_id,visit_id,completed,n_back_jumps,outcome,t_total
9405,4079649,135423080_71753269572,155993046_1538887816_806756,1,0,successful,3.816667


'Client 4033851 processes:'

,client_id,visitor_id,visit_id,completed,n_back_jumps,outcome,t_total
9301,4033851,762728880_76361333336,949661017_22392791362_127391,1,3,completed_with_errors,4.983333


'Client 5305116 processes:'

,client_id,visitor_id,visit_id,completed,n_back_jumps,outcome,t_total
12270,5305116,48895020_52187827152,454423043_73789478140_859004,0,0,unsuccessful,NaN
12271,5305116,87196875_82592805389,69164930_20082199445_40666,1,0,successful,4.8
